<a href="https://colab.research.google.com/github/marina554/Consolidated-Accounting/blob/main/%E6%8A%95%E8%B3%87%E3%81%A8%E8%B3%87%E6%9C%AC%E3%81%AE%E6%B6%88%E5%8E%BB(Investment_Elimination).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =========================================
# Consolidation Model (B/S + P/L + Investment Elimination)
# 連結会計モデル（貸借対照表＋損益計算書＋投資と資本の消去）
# =========================================
import pandas as pd

# ---------------------------
# 1. Individual Financial Data
# 単体財務データの準備
# ---------------------------

# Parent company 親会社
parent_bs = {
    "Cash": 1000,
    "Accounts Receivable": 500,
    "Inventory": 300,
    "Investment in Subsidiary": 300,  # 子会社への投資（消去対象）
    "Accounts Payable": 400,
    "Intercompany AR": 200,  # 親子間売掛金
    "Capital Stock": 1000,
    "Retained Earnings": 700
}

parent_pl = {
    "Sales": 2000,
    "Cost of Goods Sold": 800,
    "SG&A": 300,
    "Intercompany Sales": 200,  # 親子間売上
}

# Subsidiary company 子会社
child_bs = {
    "Cash": 300,
    "Accounts Receivable": 100,
    "Inventory": 100,
    "Intercompany AP": 200,  # 親子間買掛金
    "Accounts Payable": 150,
    "Capital Stock": 200,
    "Retained Earnings": 150
}

child_pl = {
    "Sales": 500,
    "Cost of Goods Sold": 200,
    "SG&A": 100,
    "Intercompany Purchases": 200  # 親子間仕入
}

# Ownership ratio 持分比率
ownership = 0.6  # 親会社が60%所有

# ---------------------------
# 2. Eliminate Intercompany Transactions
# 親子間取引の消去
# ---------------------------
def eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs):
    # 売上・仕入の消去
    internal_amount = min(parent_pl["Intercompany Sales"], child_pl["Intercompany Purchases"])
    parent_pl["Intercompany Sales"] -= internal_amount
    child_pl["Intercompany Purchases"] -= internal_amount

    # 売掛金・買掛金の消去
    internal_bs_amount = min(parent_bs["Intercompany AR"], child_bs["Intercompany AP"])
    parent_bs["Intercompany AR"] -= internal_bs_amount
    child_bs["Intercompany AP"] -= internal_bs_amount

    print(f"Eliminated intercompany sales/purchases: {internal_amount}  親子間売上・仕入の消去")
    print(f"Eliminated intercompany receivables/payables: {internal_bs_amount}  親子間債権・債務の消去")

eliminate_internal_transactions(parent_pl, child_pl, parent_bs, child_bs)

# ---------------------------
# 3. Investment and Equity Elimination
# 投資と資本の相殺消去
# ---------------------------
def eliminate_investment_equity(parent_bs, child_bs, ownership):
    """
    Eliminate parent's investment in subsidiary against subsidiary's equity.
    親会社の子会社投資を子会社の資本と相殺
    """
    sub_equity = child_bs["Capital Stock"] + child_bs["Retained Earnings"]  # 子会社資本
    parent_investment = parent_bs["Investment in Subsidiary"]  # 親会社の投資

    # 親会社持分と非支配株主持分の計算
    parent_share = sub_equity * ownership
    nci_share = sub_equity * (1 - ownership)

    # 相殺処理
    parent_bs["Investment in Subsidiary"] -= parent_investment
    child_bs["Capital Stock"] -= child_bs["Capital Stock"]
    child_bs["Retained Earnings"] -= child_bs["Retained Earnings"]

    print(f"Investment elimination: {parent_investment}  投資と資本の消去額")
    print(f"Parent share of sub equity: {parent_share}  親会社持分")
    print(f"Non-controlling interest (NCI): {nci_share}  非支配株主持分")

    return nci_share  # 連結B/Sに使用するNCI額

# 相殺実行
nci_equity = eliminate_investment_equity(parent_bs, child_bs, ownership)

# ---------------------------
# 4. Create Consolidated B/S
# 連結貸借対照表作成
# ---------------------------
def create_consolidated_bs(parent, child, nci_equity):
    consolidated = {}
    for key in set(parent.keys()) | set(child.keys()):
        consolidated[key] = parent.get(key, 0) + child.get(key, 0)
    consolidated["Non-controlling Interests"] = nci_equity  # 非支配株主持分を追加
    return consolidated

# ---------------------------
# 5. Create Consolidated P/L
# 連結損益計算書作成
# ---------------------------
def create_consolidated_pl(parent_pl, child_pl, ownership):
    consolidated = {}
    for key in set(parent_pl.keys()) | set(child_pl.keys()):
        consolidated[key] = parent_pl.get(key, 0) + child_pl.get(key, 0)

    # 親子間取引は削除
    consolidated.pop("Intercompany Sales", None)
    consolidated.pop("Intercompany Purchases", None)

    # 営業利益の計算
    consolidated["Operating Income"] = (
        consolidated["Sales"]
        - consolidated["Cost of Goods Sold"]
        - consolidated["SG&A"]
    )

    # 子会社純利益と非支配株主持分計算
    child_net_income = child_pl["Sales"] - child_pl["Cost of Goods Sold"] - child_pl["SG&A"]
    nci_income = child_net_income * (1 - ownership)

    consolidated["Net Income Attributable to NCI"] = nci_income  # 非支配株主持分
    consolidated["Net Income Attributable to Parent"] = consolidated["Operating Income"] - nci_income  # 親会社帰属

    return consolidated

# ---------------------------
# 6. Run Consolidation
# 連結計算実行
# ---------------------------
consolidated_bs = create_consolidated_bs(parent_bs, child_bs, nci_equity)
consolidated_pl = create_consolidated_pl(parent_pl, child_pl, ownership)

df_bs = pd.DataFrame([consolidated_bs], index=["Consolidated B/S"])
df_pl = pd.DataFrame([consolidated_pl], index=["Consolidated P/L"])

print("\n==== CONSOLIDATED BALANCE SHEET (B/S) ====")
print(df_bs.T)

print("\n==== CONSOLIDATED INCOME STATEMENT (P/L) ====")
print(df_pl.T)


Eliminated intercompany sales/purchases: 200  親子間売上・仕入の消去
Eliminated intercompany receivables/payables: 200  親子間債権・債務の消去
Investment elimination: 300  投資と資本の消去額
Parent share of sub equity: 210.0  親会社持分
Non-controlling interest (NCI): 140.0  非支配株主持分

==== CONSOLIDATED BALANCE SHEET (B/S) ====
                           Consolidated B/S
Retained Earnings                     700.0
Cash                                 1300.0
Capital Stock                        1000.0
Investment in Subsidiary                0.0
Intercompany AR                         0.0
Intercompany AP                         0.0
Inventory                             400.0
Accounts Payable                      550.0
Accounts Receivable                   600.0
Non-controlling Interests             140.0

==== CONSOLIDATED INCOME STATEMENT (P/L) ====
                                   Consolidated P/L
SG&A                                          400.0
Sales                                        2500.0
Cost of Goods Sold   